In [46]:
import torch
import torch.optim as optim
import torch.nn.functional as F

lr = 50
smoothing = 0.1
epochs = 500
g = torch.Generator().manual_seed(42)
model = Classifier()

# create an optimizer that works best with cross entropy loss and label smoothing
optimizer = optim.SGD([model.W], lr=lr)

loss_fn = torch.nn.CrossEntropyLoss(reduction='mean')

words = open('names.txt').read().splitlines()
letters = sorted(list(set(''.join(words))))
letter_to_index = {letter: index for index, letter in enumerate(letters)}
letter_to_index['.'] = 0
index_to_letter = {i: letter for letter, i in letter_to_index.items()}

xs_train, ys_train = [], []
xs_train_num = 0

trainValTestSplit = [0.8, 0.1, 0.1]

indices = torch.randperm(len(words), generator=g)

for i in indices[:int(len(indices) * trainValTestSplit[0])]:
    chs = ['.'] + list(words[i]) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        xs_train.append(letter_to_index[ch1])
        ys_train.append(letter_to_index[ch2])

xs_train = torch.tensor(xs_train)
ys_train = torch.tensor(ys_train)

xs_train_num = xs_train.nelement()
xs_train_encoded = torch.nn.functional.one_hot(xs_train, num_classes=27).float()

for epoch in range(epochs):
    model.train()
    logits = model(xs_train_encoded)

    # compute the loss using pytorch crossentropyloss
    loss = loss_fn(logits, ys_train) + 0.01 * model.W.pow(2).mean()
    print(loss.item())
    loss.backward()

    optimizer.step()

    optimizer.zero_grad()



3.879657030105591
3.3567168712615967
3.0779082775115967
2.912031888961792
2.822808265686035
2.7493796348571777
2.7053730487823486
2.6590123176574707
2.633002758026123
2.5994739532470703
2.583867073059082
2.5580544471740723
2.5490293502807617
2.528029680252075
2.523172616958618
2.505286455154419
2.5032098293304443
2.487494707107544
2.4874110221862793
2.473306179046631
2.474717617034912
2.4618453979492188
2.464406967163086
2.4524900913238525
2.4559431076049805
2.4447708129882812
2.4489171504974365
2.4383249282836914
2.4430124759674072
2.432872772216797
2.4379851818084717
2.42820143699646
2.433649778366089
2.424149513244629
2.4298675060272217
2.4205963611602783
2.42653489112854
2.417452096939087
2.4235739707946777
2.4146485328674316
2.420924425125122
2.412132501602173
2.4185397624969482
2.4098622798919678
2.4163830280303955
2.407804489135742
2.414423942565918
2.4059319496154785
2.4126381874084473
2.404221773147583
2.411004066467285
2.4026544094085693
2.4095044136047363
2.401214361190796
2

In [39]:
# create a classifier class that inherits from nn.Module
class Classifier(torch.nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        g = torch.Generator().manual_seed(42)
        self.W = torch.nn.Parameter(torch.randn((27,27), generator=g, requires_grad=True))

    def forward(self, x):
        return x @ self.W

In [47]:
model.state_dict()

OrderedDict([('W',
              tensor([[ 1.2800,  0.4038,  0.4746,  0.7985,  0.5629, -0.8016, -0.4179,  0.9675,
                        0.5786,  0.7591,  1.0589,  1.2164,  1.2261,  1.7896, -0.9987, -0.7177,
                       -1.9933,  1.4172,  0.9370,  0.4702, -0.9856, -0.0156, -0.9470, -1.3609,
                        0.7320,  0.0858, -3.8898],
                      [ 2.7511,  0.4139, -1.0662,  0.8373,  3.1582, -1.3292, -1.2371,  0.3299,
                        2.0671, -1.0457, -1.1760,  1.2340, -1.2224, -1.1538,  1.2838, -1.2384,
                       -1.1981,  3.4357, -0.7637, -1.2301,  0.4117, -1.2438, -1.2058, -1.1255,
                        1.1684, -1.1836, -1.0635],
                      [ 3.2861, -1.3133,  0.0918, -1.1781,  2.7966, -1.2020, -1.1281,  3.0031,
                        2.0582, -1.0109,  2.2320,  1.0979, -1.1661, -1.2090,  2.4309, -1.1956,
                       -0.8673,  0.6790, -0.9613, -0.0375,  0.0223, -1.2140, -1.2065, -1.1764,
                        

In [50]:
%run -i data_preparation/v0.py